<a href="https://colab.research.google.com/github/NUG30/homework-2-minhdiep1204/blob/main/Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 2: Naive Bayes Email Spam Filter

In this homework, we will learn how to implement the Naive Bayes classifier in order to create a simple email spam filter. This spam filter will be trained by test_emails, which will be given by a vector of tuples (emails, spam/nospam). For each tuple the first entry is a string ("email"), and the second entry is 0 or 1, depending whether the email contains spam words.

In [1]:
dictionary = ['hello', 'students', 'please', 'learn', 'for', 'the', 'exam', 'buy', 'drugs', 'today', 'sun', 'is', 'shining', 'in', 'nagoya', 'lets', 'sell', 'how', 'are', 'you', 'today?', 'do', 'your', 'homework', 'want', 'free', 'solutions?', 'hey', 'always', 'ask', 'questions', 'if','have', 'any', 'math', 'not', 'good', 'submit', 'pay'] 

test_emails=[
             ["hello students, please learn for the exam", 0],
             ["hello students, please buy drugs", 1],    
             ["hello, today the sun is shining in nagoya", 0],
             ["lets sell drugs in nagoya", 1],
             ["today learn drugs", 1],
             ["how are you today?", 0],
             ["hello students, please do your homework", 0],
             ["hello, do you want free homework solutions?", 1],
             ["hey students, please always ask questions if you have any", 0],
             ["math is not good", 1],
             ["math is good", 0],
             ["please submit your homework", 0],
             ["please buy questions", 1],
             ["please pay for the exam", 1]          
             ]

The feature space for our spam filter will be $\mathcal{X}=\{0,1\}^d$, where $d$ denotes the number of words in the dictionary. For a feature (email) $x \in \mathcal{X}$ the entry $x_i$ for $i=1,\dots,d$ is $1$ if the $i$-th word of the dictionary is contained in the email and $0$ otherwise.  

# **Exercise 1**
Implement a function which creates a feature vector out of an email and a function which creates a training set out of test emails. 

You would need to figure out whether a sentence contains a word, and there are functions in Python that could determine whether a string contains another string. You can consult documentation (and Google) to find out.

In [ ]:
import numpy as np
def email_to_feature(dict, email):
  d = len(dict)
  x = np.zeros(d)
  for i in range(d):
    if dict[i] in email:
      x[i] = 1
  return x

print(email_to_feature(dictionary, test_emails[1][0]))

def emails_to_test_samples(dict, test_emails):
  T = test_emails
  n = len (test_emails)
  for i in range (n):
    T[i][0] = email_to_feature(dict, test_emails[i][0])
  return T

print (emails_to_test_samples(dictionary, test_emails))

[1. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[[array([1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.]), 0], [array([1., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.]), 1], [array([1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.]), 0], [array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.]), 1], [array([0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

 **Recall from Lecture 6:**

Given a training set  $\mathcal{T} = \left( (x^{(1)}, y^{(1)}) , \dots, (x^{(n)}, y^{(n)})   \right)$ we calculate for $i=1,\dots,d$ the following numbers
\begin{align*}
\phi_{i\mid y=1} &= \frac{1+\sum_{j=1}^n I(x^{(j)}_i = 1  \,\wedge \, y^{(j)}=1 ) }{2+\sum_{j=1}^n I(y^{(j)}=1)}\,,\\
	\phi_{i\mid y=0} &= \frac{1+\sum_{j=1}^n I(x^{(j)}_i = 1  \,\wedge \, y^{(j)}=0 )}{2+\sum_{j=1}^n I(y^{(j)}=0)}\,,\\
		\phi_{y=1} &= \frac{1+\sum_{j=1}^n I(y^{(j)} = 1)}{2+n} \,.
\end{align*}
Here $I$ denoted the indicator function. We used the laplace smoothing (thats why we have the $1+$ and $2+$) in order to make sure that we will not assume probability $0$ for unknown words.

Now assume we get a new feature (i.e. someone sends us an email) $x \in \{0,1\}^d$. Then we can calculate for each word $i=1,\dots,d$ the probabilities
\begin{align*}
P(x_i = 1 \mid y=1) &= \phi_{i\mid y=1}\,,\qquad &&P(x_i = 1 \mid y=0) = \phi_{i\mid y=0} \,,\\
P(x_i = 0 \mid y=1) &= 1- \phi_{i\mid y=1}\,,\qquad &&P(x_i = 0 \mid y=0) = 1-\phi_{i\mid y=0} \,. \\
\end{align*}

By the Naive Bayes assumption we have for $x \in \{0,1\}^d$
\begin{align*}
P(x \mid y)  = \prod_{i=1}^d P(x_j \mid y)\,.
\end{align*}
The probability of the new email being spam is then
\begin{align*}
P(y=1 \mid x) &= \frac{ P(x \mid y=1) P(y=1)}{P(x)}\\
&= \frac{\prod_{i=1}^d P(x_i \mid y = 1) \cdot \phi_{y=1}}{\prod_{i=1}^d P(x_i \mid y = 1) \cdot \phi_{y=1} + \prod_{i=1}^d P(x_i \mid y = 0) (1-\phi_{y=1})}\,.
\end{align*}


# **Exercise 2:** 
Use the above explanation of the Naive Bayes Spam filter and implement a function which gives the probability of an email being spam given the trainings email above. 


In [ ]:
# You can implement the above algorithm in any way you want.
# One possible way:
# - Calculate the phis here
# - Write functions for the probability P(x|y) depending on these phis
# - Use this function in the function spam_percentage 

#calculate phis 
def phi_y1 (T):
  num = 1
  denum = 2
  for j in range(len(T)): 
    if T[j][1] == 1:
      num += 1
      denum = 2 + len(T)
  return num/denum

def phi_x_y1 (T, dict):
  num = 1
  denum = 2
  phi = np.zeros(len(dict))
  for i in range(len(dict)):
    for j in range(len(T)):
      if T[j][1] == 1 and T[j][0][i] == 1:
        num +=1
      if T[j][1] == 1:
        denum +=1
    phi[i] = num/denum
  return phi

def phi_x_y0 (T, dict):
  num = 1
  denum = 2
  phi = np.zeros(len(dict))
  for i in range(len(dict)):
    for j in range(len(T)):
      if T[j][1] == 0 and T[j][0][i] == 1:
        num +=1
      if T[j][1] == 1:
        denum +=1
    phi[i] = num/denum
  return phi

T = emails_to_test_samples(dictionary, test_emails)
phi_y1 = phi_y1 (T)
phi_x_y1 = phi_x_y1 (T, dictionary)
phi_x_y0 = phi_x_y0 (T, dictionary)
print (phi_y1) 
print (phi_x_y1)
print (phi_x_y0)

def prob_y1 (x):
  p = 1
  for i in range(len(x)):
    if x[i] == 1:
      p *= phi_x_y1[i]
    else:
      p *= 1 - phi_x_y1[i]
  return p 

def prob_y0 (x):
  p = 1
  for i in range(len(x)):
    if x[i] == 1:
      p *= phi_x_y0[i]
    else:
      p *= 1 - phi_x_y0[i]
  return p 

def spam_percentage(email):
  x = email_to_feature(dictionary, email)
  num = denum = 0
  num += prob_y1(x)*phi_y1
  denum += num + prob_y0(x)*(1-phi_y1)
  return num/denum 


0.5
[0.11111111 0.0625     0.04347826 0.03333333 0.02702703 0.02272727
 0.01960784 0.01724138 0.01538462 0.01388889 0.01265823 0.01162791
 0.01075269 0.01       0.00934579 0.00877193 0.00826446 0.0078125
 0.00740741 0.00704225 0.00671141 0.00641026 0.00613497 0.00588235
 0.00564972 0.00543478 0.0052356  0.00505051 0.00487805 0.00471698
 0.00456621 0.00442478 0.00429185 0.00416667 0.00404858 0.00393701
 0.00383142 0.00373134 0.00363636]
[0.11111111 0.0625     0.04347826 0.03333333 0.02702703 0.02272727
 0.01960784 0.01724138 0.01538462 0.01388889 0.01265823 0.01162791
 0.01075269 0.01       0.00934579 0.00877193 0.00826446 0.0078125
 0.00740741 0.00704225 0.00671141 0.00641026 0.00613497 0.00588235
 0.00564972 0.00543478 0.0052356  0.00505051 0.00487805 0.00471698
 0.00456621 0.00442478 0.00429185 0.00416667 0.00404858 0.00393701
 0.00383142 0.00373134 0.00363636]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  


Test your spam filter with the following email.


In [ ]:
email="the sun is shining. buy drugs now"
email1 = "i want to go home"
print(spam_percentage(email))

0.5



# **Exercise 3**
Extend your spamfilter by creating a dynamical dictionary. Instead of starting with a fixed dictionary, you should now create a dictionary out of a list of emails. 

Write a function `create_dictionary(emails)` which returns a dictionary created from a list of emails (Give as an array of arrays `[text, spam\nospam]`). Make sure that you do not include words more than once into the dictionary.
To implement this function you should look up the function `split()` for a string in Python. To take care of the symbols "." and "," you can use the `replace()` function of a string.

In [4]:
def create_dictionary(emails):
  D = []
  for j in range (len(emails)):
    D += emails[j][0].split()
  return D

print (create_dictionary(test_emails))

['hello', 'students,', 'please', 'learn', 'for', 'the', 'exam', 'hello', 'students,', 'please', 'buy', 'drugs', 'hello,', 'today', 'the', 'sun', 'is', 'shining', 'in', 'nagoya', 'lets', 'sell', 'drugs', 'in', 'nagoya', 'today', 'learn', 'drugs', 'how', 'are', 'you', 'today?', 'hello', 'students,', 'please', 'do', 'your', 'homework', 'hello,', 'do', 'you', 'want', 'free', 'homework', 'solutions?', 'hey', 'students,', 'please', 'always', 'ask', 'questions', 'if', 'you', 'have', 'any', 'math', 'is', 'not', 'good', 'math', 'is', 'good', 'please', 'submit', 'your', 'homework', 'please', 'buy', 'questions', 'please', 'pay', 'for', 'the', 'exam']
